# Kaggle 快速回归（200样本、epoch=2）

参考 `KAGGLE_DEPLOYMENT_GUIDE.md`：检查/复制代码、安装依赖、生成配置、运行、收集并打包结果。

In [ ]:
import os, shutil
from pathlib import Path

# 1) 定位代码数据集
candidates = [
    Path('/kaggle/input/mcm-project/MCM'),
    Path('/kaggle/input/mcm-project'),
    Path('/kaggle/input/mcm-ner-training/MCM'),
    Path('/kaggle/input/mcm-ner-training'),
]
code_src = next((p for p in candidates if p.exists()), None)
if code_src is None:
    raise FileNotFoundError('未找到项目数据集，请在Data面板添加 mcm-project 或同名数据集')
print('✓ 代码源目录:', code_src)

# 2) 复制到可写目录 /kaggle/working/MCM
work_dir = Path('/kaggle/working/MCM')
if work_dir.exists():
    shutil.rmtree(work_dir)
shutil.copytree(code_src, work_dir)
os.chdir(work_dir)
print('✓ 已复制到工作目录:', work_dir)

# 可选：列出关键文件
print('项目根内容示例:', sorted([p.name for p in work_dir.iterdir()])[:10])

In [ ]:
# 3) 安装依赖（尽量精简，优先 requirements_kaggle.txt ）
!pip install -q -r requirements_kaggle.txt

In [ ]:
import json, subprocess, sys
from pathlib import Path

# 4) 生成快速回归配置（200样本、epoch=2、batch=4）
methods = ["none", "replay", "ewc", "si", "gem", "mas", "moe_adapters", "lwf", "tam_cl"]
cmd = [sys.executable, "scripts/generate_kaggle_quick_test_configs.py", "--output_dir", "scripts/configs/kaggle_quick_test"] + ["--methods"] + methods
subprocess.check_call(cmd)

cfg_dir = Path("scripts/configs/kaggle_quick_test")
cfgs = sorted(cfg_dir.glob("*/*.json"))
print('✓ 配置文件数:', len(cfgs))
print('示例:', [c.as_posix() for c in cfgs][:5])

In [ ]:
# 5) 逐个运行配置
for cfg in cfgs:
    print(f"\n=== Running {cfg} ===")
    subprocess.check_call([sys.executable, "-m", "scripts.train_with_zero_shot", "--config", str(cfg)])

In [ ]:
import zipfile

# 6) 收集并打包结果
result_dir = Path("/kaggle/working/quick_test_results")
result_dir.mkdir(parents=True, exist_ok=True)

# 拷贝关键输出（train_info、热力图、模型等）
src_dirs = [
    Path("checkpoints/kaggle_quick"),
    Path("checkpoints/quick_test")
]
for sd in src_dirs:
    if sd.exists():
        dest = result_dir / sd.name
        if dest.exists():
            shutil.rmtree(dest)
        shutil.copytree(sd, dest)
        print('✓ 拷贝', sd, '->', dest)

zip_path = Path('/kaggle/working/quick_test_results.zip')
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
    for p in result_dir.rglob('*'):
        zf.write(p, p.relative_to(result_dir))
print('✓ 打包完成:', zip_path)
print('可在 Kaggle 右侧“文件”面板下载 quick_test_results.zip')